<a href="https://colab.research.google.com/github/JeremySpradlin/DS17-Unit-2-Build-Project/blob/master/notebooks/Jeremy_Spradlin_DS17_Unit_2_Build_week.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jeremy Spradlin Unit 2 Build Week

This is the start of my notebook for my DS17 Unit 2 Build week.

This project will be concatenating numerous data sources of solar weather and solar weather conected measurements in order to predict `Insert Target Here`.


# Imports
Below will be where we import packages for use in this notebook

In [2]:
#Imports

import pandas as pd
import numpy as np
import plotly.express as px


# Getting Our Data Sets

Below we will import our datasets

In [3]:
#Sun Spot Data
url1 = 'https://raw.githubusercontent.com/JeremySpradlin/DS-Unit-2-Build-Week/master/sunspot_data.csv'

#Solar Flares
url2 = 'https://raw.githubusercontent.com/JeremySpradlin/DS-Unit-2-Build-Week/master/hessi.solar.flare.2002to2016.csv'

#Earth Surface Temperatures
url3 = 'https://raw.githubusercontent.com/JeremySpradlin/DS-Unit-2-Build-Week/master/temperature_data.csv'

#Radio Flux
url4 = 'https://raw.githubusercontent.com/JeremySpradlin/DS-Unit-2-Build-Week/master/radio_flux.csv'

sspot = pd.read_csv(url1)
sflare = pd.read_csv(url2)
temp = pd.read_csv(url3)
flux = pd.read_csv(url4)

Below we will clean each portion of data and work to normlize time across the datasets using fraction of a date format, eg year.fraction_of_year

# Sun spot EDA

In [4]:
#Looking at our initial data
sspot.head()

,Unnamed: 0,Year,Month,Day,Date In Fraction Of Year,Number of Sunspots,Standard Deviation,Observations,Indicator
0,0,1818,1,1,1818.001,-1,-1.0,0,1
1,1,1818,1,2,1818.004,-1,-1.0,0,1
2,2,1818,1,3,1818.007,-1,-1.0,0,1
3,3,1818,1,4,1818.010,-1,-1.0,0,1
4,4,1818,1,5,1818.012,-1,-1.0,0,1


In [5]:
#Create our data wrangling function

def wrangle_sspots(df):
  """This function will take in a dataframe of Sunspot activity
  and perform different functions and actions on it to 
  prepare the dataset for training in a predictive model."""

  #Remove spaces from column names and change to lowercase
  df.columns = df.columns.str.lower().str.replace(' ', '_')

  #Replace -1's in target column
  df['number_of_sunspots'].replace(-1, np.NaN, inplace=True)

  #Remove observations with missing values or no observations
  df = df.dropna()
  mask = df[(df['observations'] == 0)].index
  df = df.drop(mask)

  #Create our datetime column
  df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

    #Remove columns
  df = df.drop(['indicator', 'unnamed:_0', 'year', 'month', 'day', 'date_in_fraction_of_year'], axis=1)

  #move the date column to the front
  col = df.pop('date')
  df.insert(0, 'date', col)

  #Reset Index
  df.reset_index(drop=True, inplace=True)

  return df

In [6]:
sspot = wrangle_sspots(sspot)
sspot.head()

,date,number_of_sunspots,standard_deviation,observations
0,1818-01-08,65.0,10.2,1
1,1818-01-13,37.0,7.7,1
2,1818-01-17,77.0,11.1,1
3,1818-01-18,98.0,12.6,1
4,1818-01-19,105.0,13.0,1


# Solar Flare EDA

In [7]:
sflare.head()

,flare,start.date,start.time,peak,end,duration.s,peak.c/s,total.counts,energy.kev,x.pos.asec,y.pos.asec,radial,active.region.ar,flag.1,flag.2,flag.3,flag.4,flag.5
0,2021213,2002-02-12,21:29:56,21:33:38,21:41:48,712,136,167304,12-25,592,-358,692,0,A1,P1,NaN,NaN,NaN
1,2021228,2002-02-12,21:44:08,21:45:06,21:48:56,288,7,9504,6-12,604,-341,694,9811,A1,P1,PE,Q1,NaN
2,2021332,2002-02-13,00:53:24,00:54:54,00:57:00,216,15,11448,6-12,-310,375,487,9825,A1,P1,NaN,NaN,NaN
3,2021308,2002-02-13,04:22:52,04:23:50,04:26:56,244,20,17400,12-25,-277,378,469,9822,A1,P1,NaN,NaN,NaN
4,2021310,2002-02-13,07:03:52,07:05:14,07:07:48,236,336,313392,25-50,-272,390,476,9825,A1,GS,P1,PE,Q2


In [8]:
sflare['start.date']

0         2002-02-12
1         2002-02-12
2         2002-02-13
3         2002-02-13
4         2002-02-13
             ...    
113937    2016-12-26
113938    2016-12-26
113939    2016-12-27
113940    2016-12-27
113941    2016-12-27
Name: start.date, Length: 113942, dtype: object

Looking at the dataset above, we can see that there are often several solar flares on the same day.  we will need to engineer at least one new feature:

- Create a column that contains the date in a fraction of a year

In addition to this, we will need to account for the fact that multiple flares can occur in the same day period.

# Temperature EDA

In [9]:
temp.head()

,"Date Number,","Year,","Month,","Day,","Day of Year,",Anomaly
0,NaN,NaN,NaN,NaN,NaN,NaN
1,1880.001,1880.0,1.0,1.0,1.0,0.296
2,1880.004,1880.0,1.0,2.0,2.0,0.953
3,1880.007,1880.0,1.0,3.0,3.0,1.222
4,1880.010,1880.0,1.0,4.0,4.0,1.417


Our Temperature dataset already has a date number in fraction form, so this one is already good to join to the other datasets

In [10]:
#Create our Wrangle Function for our temperature dataset
def wrangle_temp(df):
  """This dataset will take in our temperature data set, clean it, and prepare it for 
  easily joining with the other datasets"""

  #Drop the blank top row
  df.drop(index=0, inplace=True)

  #Fix our column names
  df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace(',', '')

  #Create our date column
  df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

  #move the date column to the front
  col = df.pop('date')
  df.insert(0, 'date', col)

  #Reset Index
  df.reset_index(drop=True, inplace=True)

  #Drop extra columns that are not needed
  df = df.drop(['date_number', 'year', 'month', 'day', 'day_of_year'], axis=1)

  return df

In [11]:
temp = wrangle_temp(temp)
temp.head()

,date,anomaly
0,1880-01-01,0.296
1,1880-01-02,0.953
2,1880-01-03,1.222
3,1880-01-04,1.417
4,1880-01-05,1.119


# Radio Flux EDA

In [12]:
flux.head()

,year m,onth,day,cont_day,f30,f30_c,f30_p,f30_f,f15,f15_c,f15_p,f15_f,f10.7,f10.7_c,f10.7_p,f10.7_f,f8,f8_c,f8_p,f8_f,f3.2,f3.2_c,f3.2_p,f3.2_f
0,1951,11,1,669,71.4,71.4,1.9,2,87.6,87.6,2.5,2,107.3,107.3,3.6,0,110.6,110.6,3.1,2,288.9,288.9,3.5,2
1,1951,11,2,670,70.3,70.3,1.9,2,85.6,85.6,2.5,2,104.7,104.7,3.6,0,108.3,108.3,3.1,2,287.1,287.1,3.5,2
2,1951,11,3,671,69.5,69.5,1.9,2,84.7,84.7,2.5,2,106.1,106.1,3.6,2,107.7,107.7,3.0,2,287.0,287.0,3.5,2
3,1951,11,4,672,66.8,66.8,1.9,2,81.1,81.1,2.5,2,107.2,107.2,3.6,0,105.3,105.3,3.0,2,286.8,286.8,3.5,2
4,1951,11,5,673,71.8,71.8,1.9,2,88.4,88.4,2.5,2,114.0,114.0,3.6,0,111.3,111.3,3.0,2,288.1,288.1,3.5,2


These are daily values, and so we will need to expand our multi-daily values to account for solar flares which can occur multiple times per day.

We will need to engineer the following new feature:

- A column that contains the date in fraction of year

In [13]:
#Create our Wrangle function for our radio flux

def wrangle_flux(df):
  """ This function will take in the radio flux dataset and clean
  and adjust the data as needed for joining with the other datasets"""

  #Clean column names
  df = df.rename(columns={
    'year m': 'year',
    'onth': 'month',
  })

  #Create our date column
  df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

  #Drop extra columns
  df = df.drop(['year', 'month', 'day', 'cont_day'], axis=1)

  #move our date column to the front
  col = df.pop('date')
  df.insert(0, 'date', col)

  #Reset Index
  df.reset_index(drop=True, inplace=True)

  return df


In [14]:
flux = wrangle_flux(flux)
flux.head()

,date,f30,f30_c,f30_p,f30_f,f15,f15_c,f15_p,f15_f,f10.7,f10.7_c,f10.7_p,f10.7_f,f8,f8_c,f8_p,f8_f,f3.2,f3.2_c,f3.2_p,f3.2_f
0,1951-11-01,71.4,71.4,1.9,2,87.6,87.6,2.5,2,107.3,107.3,3.6,0,110.6,110.6,3.1,2,288.9,288.9,3.5,2
1,1951-11-02,70.3,70.3,1.9,2,85.6,85.6,2.5,2,104.7,104.7,3.6,0,108.3,108.3,3.1,2,287.1,287.1,3.5,2
2,1951-11-03,69.5,69.5,1.9,2,84.7,84.7,2.5,2,106.1,106.1,3.6,2,107.7,107.7,3.0,2,287.0,287.0,3.5,2
3,1951-11-04,66.8,66.8,1.9,2,81.1,81.1,2.5,2,107.2,107.2,3.6,0,105.3,105.3,3.0,2,286.8,286.8,3.5,2
4,1951-11-05,71.8,71.8,1.9,2,88.4,88.4,2.5,2,114.0,114.0,3.6,0,111.3,111.3,3.0,2,288.1,288.1,3.5,2


#Joining Data Sets

In [16]:
df = pd.merge(sspot, temp, how='inner', on='date')
df.head()

,date,number_of_sunspots,standard_deviation,observations,anomaly
0,1880-01-01,0.0,2.0,1,0.296
1,1880-01-02,7.0,2.9,1,0.953
2,1880-01-03,28.0,6.0,1,1.222
3,1880-01-04,55.0,8.3,1,1.417
4,1880-01-05,55.0,8.3,1,1.119


In [17]:
df = pd.merge(df, flux, how='inner', on='date')
df.head()

,date,number_of_sunspots,standard_deviation,observations,anomaly,f30,f30_c,f30_p,f30_f,f15,f15_c,f15_p,f15_f,f10.7,f10.7_c,f10.7_p,f10.7_f,f8,f8_c,f8_p,f8_f,f3.2,f3.2_c,f3.2_p,f3.2_f
0,1951-11-01,64.0,5.3,1,-1.914,71.4,71.4,1.9,2,87.6,87.6,2.5,2,107.3,107.3,3.6,0,110.6,110.6,3.1,2,288.9,288.9,3.5,2
1,1951-11-02,81.0,6.0,1,-2.271,70.3,70.3,1.9,2,85.6,85.6,2.5,2,104.7,104.7,3.6,0,108.3,108.3,3.1,2,287.1,287.1,3.5,2
2,1951-11-03,65.0,5.4,1,-1.389,69.5,69.5,1.9,2,84.7,84.7,2.5,2,106.1,106.1,3.6,2,107.7,107.7,3.0,2,287.0,287.0,3.5,2
3,1951-11-04,75.0,5.8,1,-0.913,66.8,66.8,1.9,2,81.1,81.1,2.5,2,107.2,107.2,3.6,0,105.3,105.3,3.0,2,286.8,286.8,3.5,2
4,1951-11-05,86.0,6.2,1,-0.900,71.8,71.8,1.9,2,88.4,88.4,2.5,2,114.0,114.0,3.6,0,111.3,111.3,3.0,2,288.1,288.1,3.5,2


In [18]:
df.shape

(24806, 25)

In [21]:
px.scatter(df, x='date', y='number_of_sunspots', color='f30_c')